# Résoudre les élisions fautives

## L’élision en français

**Élision :** mécanisme d’effacement de la voyelle finale d’un mot devant la voyelle du mot suivant, matérialisée par une apostrophe (`’`).

Phénomène courant, parfois obligatoire (1), facultatif (2) ou totalement fautif (3) :
```txt
(1) *Le arbre est grand.
(2) Arthur : Et dis donc machin, tu fais péter du raisin,
    t'es gentil !
(3) Le tavernier : Ah heu Sire, j'vais pas vous raconter
    d'conneries.
```

**Rappel :** la retranscription des scripts de la série ne suit aucune directive. Volonté des transcripteurs de reproduire à l’écrit ce qu’ils entendent à l’oral.

**Objectif :** rétablir les formes improprement élidées (4).
```txt
(4) Le tavernier : Ah heu Sire, je vais pas vous raconter
    de conneries.
```

## Méthodologie

Détecter les élisions fautives avant de les résoudre !

Recours à une expression rationnelle pour détecter les motifs non autorisés.

**Motif autorisé :** un mot suivi d’une apostrophe qui soit lui-même suivi d’un mot débutant par une voyelle.  
**Motif non autorisé :** un mot suivi d’une apostrophe qui **ne** soit lui-même **pas** suivi d’un mot débutant par une voyelle.

Exemple de détection des élisions autorisées :

In [ ]:
import re

# Pattern to detect elisions
allowed = re.compile(r'\w+[\'’][aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+')

# Reading a file
with open(f'./sample/S01E01-heat.txt') as file:
    text = file.read()

results = allowed.findall(text)
print(results)

Pour détecter les élisions fautives, modifier légèrement la *regex* :

In [ ]:
wrong = re.compile(r'\w+[\'’][^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+')
results = wrong.findall(text)
print(results)

Signe `^` pour exclure les caractères autorisés après une élision.

Pour résoudre les élisions : transformer le signe `'` en `e` et rajouter une espace.

Utiliser les parenthèses pour capturer dans la *regex* des parties à réutiliser :

In [ ]:
pattern = re.compile(r'(\w+)[\'’]([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)')
results = pattern.findall(text)
print(results)

Pour déplier les tuples de résultats :

In [ ]:
for (elided_shape, following_word) in results:
    print(f'{elided_shape}e {following_word}', end=', ')

Quelques cas particuliers :
```txt
(5) de vant
(6) deve nir
```

Ces cas concernent les élisions internes à un mot, qui appellent un traitement manuel.

Autre considération à prendre en compte dans la méthodologie, le dédoublonnage des résultats. On comptabilise trois occurrences des formes : `s'trouve`.

Structure de données `set` pour les fusionner

In [ ]:
elisions = set()
[elisions.add(result) for result in results]
for (elided_shape, following_word) in elisions:
    print(f'{elided_shape}e {following_word}', end=', ')

En prévision, certaines élisions ambiguës :
```txt
(7) *d’mander
```

Cette forme pourrait aboutir à deux solutions :
- `demander`
- `de mander`

Les contextes gauche et droite seraient utiles pour décider. Adapter la *regex* en ce sens :

In [ ]:
pattern = re.compile(r'(\w+)\W+?(\w+)[’\']([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)\W+?(\w+)?')
results = pattern.findall(text)
for (left_context, elided_shape, following_shape, right_context) in results:
    print(f'{left_context}\t[{elided_shape} {following_shape}]\t{right_context}')

**Attention !** Conflit dans la méthodologie :
- structure `set` fusionne les résultats de la recherche sur les élisions fautives
- un résultat de cette recherche pourrait correspondre à deux réalités (*demander* vs *de mander*)

Face à une élision fautive, deux hypothèses :
- élision par suppression de la voyelle finale
- élision interne au mot

Une solution consisterait à :
1. construire un fichier des élisions fautives à résoudre
2. confronter la seconde hypothèse avec un lexique
3. traiter différemment les formes qui vérifient l’une ou l’autre des hypothèses

## Algorithme pas à pas

**Étape 1 :** importer le module `re`

In [ ]:
import re

**Étape 2 :** détecter les élisions fautives dans un texte

In [ ]:
with open(f'./sample/S01E01-heat.txt') as file:
    text = file.read()

# A list of tuples with the two parts around the elision and the left and right contexts.
pattern = re.compile(r'(\w+)\W+?(\w+)[’\']([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)\W+?(\w+)?')
results = pattern.findall(text)

**Étape 3 :** charger un lexique de référence (extrait de *Lexique 3* de Boris New et Christophe Pallier)

In [ ]:
import csv
lexicon = []
with open('./files/lexicon.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        lexicon.append(row[0])

**Étape 4 :** confronter l’hypothèse d’une élision interne avec le lexique de référence et construire une liste de motifs de remplacement.

**Remarque :** le lexique de référence étant en minuscules, il faut lui confronter la version en minuscules des hypothèses de résolution.

On note deux hypothèses de résolution face à une élision fautive :
- soit une élision interne (*d’vant* : devant)
- soit la suppression du *e* final d’un mot qui précède un mot débutant par une consonne (*s'trouve* : *se trouve*)

À la vérification de l’une ou l’autre de ces deux hypothèses s’ajoutent deux autres éventualités :
- soit les deux hypothèses se vérifient (*d’mander* : *de mander* ou *demander*)
- soit aucune de ces hypothèses ne se vérifie et alors le mot est inconnu (faute d’orthographe…)

Seul un arbitrage manuel permettra de se prononcer.

In [ ]:
# List of patterns for systematic replacements
patterns = list()
# Left and right contexts in anticipation of a manual arbitration
arbitration = ''

for (left_context, elided_shape, following_shape, right_context) in results:

    # First assumption: internal elision
    h1 = f'{elided_shape}e{following_shape}'
    # Second assumption: two separated words
    h2 = [f'{elided_shape}e', following_shape]
    
    # If both assumptions hold, manual arbitration is required
    if h1.lower() in lexicon and h2[0].lower() in lexicon and h2[1].lower() in lexicon:
        arbitration += f'{left_context}\t[{elided_shape}’{following_shape}]\t{right_context}\n'
    # Only the first assumption holds
    elif h1.lower() in lexicon:
        patterns.append((f'{elided_shape}[\'’]{following_shape}', h1))
    # Only the second assumption holds
    elif h2[0].lower() in lexicon and h2[1].lower() in lexicon:
        patterns.append((f'{elided_shape}[\'’]{following_shape}', ' '.join(h2)))
    # None of the assumptions hold: manual arbitration is required
    else:
        arbitration += f'{left_context}\t[{elided_shape}’{following_shape}]\t{right_context}\n'

**Étape 5 :** appliquer les remplacements

In [ ]:
def replace(patterns, text):
    """Replaces every instance of a pattern with its corresponding
    substitution

    patterns -- a list of tuples with the pattern and its substitution
    text -- the text in which doing the substitution
    """
    for (pattern, replacement) in patterns:
        text = re.sub(pattern, replacement, text)
    return text

text = replace(patterns, text)

**Étape 6 :** écrire un fichier recensant les contextes gauche et droite en prévision d’un arbitrage manuel de résolution des élisions fautives (*demander* vs *de mander*)

In [ ]:
# Creates the file only if there is matter to discuss
if arbitration:
    with open('./files/elisions_context.txt', 'w') as file:
        file.write('S01E01-heat\n')
        file.write(arbitration)